<a href="https://colab.research.google.com/github/CsonVass/raman/blob/main/DissolutionFromRaman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import tensorflow as tf
# tf.test.gpu_device_name()

In [2]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [3]:
# !cat /proc/cpuinfo

In [16]:
!pip install -q keras
!pip install -q scikit-learn==0.23.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [6]:
# hpmcMaps = []
# dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Raman/hpmc/c/DR_C01A-component-2.csv', header=None)


In [ ]:
map_curve_pairs = {}
missing = []
path_to_hpmc_c = '/content/drive/MyDrive/Colab Notebooks/Raman/hpmc/c'
dir_list = os.listdir(path_to_hpmc_c)
for f in dir_list: 
  path_to_map = path_to_hpmc_c + '/' + f
  map_name = f.split('-')[0].split('_')[1]
  map_curve_pairs[map_name] = {}
  map_curve_pairs[map_name]['map'] = pd.read_csv(path_to_map, header=None)

path_to_hpmc_v = '/content/drive/MyDrive/Colab Notebooks/Raman/hpmc/v'
dir_list = os.listdir(path_to_hpmc_v)
for f in dir_list: 
  path_to_map = path_to_hpmc_v + '/' + f
  map_name = f.split('-')[0].split('_')[1]
  map_curve_pairs[map_name] = {}
  map_curve_pairs[map_name]['map'] = pd.read_csv(path_to_map, header=None)

dissolutionCureves = []
curves_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Raman/Tablettak_minden_adat.csv', header=None)
names = curves_csv.iloc[0:1, 1:]
time = curves_csv.iloc[4:,0:1]
curve_points = curves_csv.iloc[4:,1:]
for i in range(len(curve_points.columns)):
    curve_i = curve_points.iloc[:, i:i+1]
    if(names.iloc[0,i] in map_curve_pairs):
      map_curve_pairs[names.iloc[0,i]]['curve'] = curve_i
    else: 
      missing.append(names.iloc[0,i])


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split



